# Chapter 01: Getting Started

This notebook explores the vangja time-series forecasting package using the classic Air Passengers dataset (similar to Facebook Prophet examples).

## Setup and Imports


In [ ]:
import warnings

warnings.filterwarnings("ignore")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from vangja import LinearTrend, FourierSeasonality
from vangja.datasets import load_air_passengers
from vangja.utils import metrics

# Set random seed for reproducibility
np.random.seed(42)

print("Imports successful!")

## 1. Load Air Passengers Dataset

The Air Passengers dataset is a classic time series dataset containing monthly totals of international airline passengers from 1949 to 1960.

Vangja provides convenience functions in `vangja.datasets` to load common datasets in the expected format (columns: `ds` for datetime, `y` for target values).

In [ ]:
# Load Air Passengers dataset using vangja.datasets
air_passengers = load_air_passengers()

print(f"Dataset shape: {air_passengers.shape}")
print(f"Date range: {air_passengers['ds'].min()} to {air_passengers['ds'].max()}")
air_passengers.head()

In [ ]:
# Visualize the data
plt.figure(figsize=(14, 5))
plt.plot(air_passengers["ds"], air_passengers["y"])
plt.title("Air Passengers Dataset")
plt.xlabel("Date")
plt.ylabel("Number of Passengers (thousands)")
plt.grid(True)
plt.show()

## 2. Train/Test Split


In [ ]:
# Split data: use last 12 months for testing
train = air_passengers[:-12].copy()
test = air_passengers[-12:].copy()

print(
    f"Training set: {train['ds'].min()} to {train['ds'].max()} ({len(train)} samples)"
)
print(f"Test set: {test['ds'].min()} to {test['ds'].max()} ({len(test)} samples)")

## 3. Model Air Passengers like Facebook Prophet

Facebook Prophet models time series as:

- Trend component (piecewise linear or logistic growth)
- Seasonality component (Fourier series)
- Holiday effects (optional)

For the Air Passengers dataset, we have:

- A clear upward trend
- Strong yearly seasonality
- Multiplicative seasonality (amplitude increases with level)


### 3.1 Additive Model


In [ ]:
# Define an additive model: Trend + Yearly Seasonality + Weekly Seasonality
model_additive = (
    LinearTrend()
    + FourierSeasonality(period=365.25, series_order=10)
    + FourierSeasonality(period=7, series_order=3)
)

print(f"Model: {model_additive}")

In [ ]:
# Fit the additive model
model_additive.fit(train)
print("Additive model fitted!")

In [ ]:
# Predict
future_additive = model_additive.predict(horizon=365, freq="D")
print(f"Predictions shape: {future_additive.shape}")
future_additive.head()

In [ ]:
# Plot results
plt.figure(figsize=(14, 5))
plt.plot(train["ds"], train["y"], "b.", label="Training data", markersize=3)
plt.plot(test["ds"], test["y"], "g.", label="Test data", markersize=3)
plt.plot(
    future_additive["ds"],
    future_additive["yhat_0"],
    "r-",
    label="Prediction",
    linewidth=1,
)
plt.title("Additive Model: Air Passengers")
plt.xlabel("Date")
plt.ylabel("Number of Passengers")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
model_additive.plot(future_additive, y_true=test)
plt.tight_layout()
plt.show()

### 3.2 Multiplicative Model

The Air Passengers data shows multiplicative seasonality (variance increases with the trend). Let's try a multiplicative model: `y = trend * (1 + seasonality)`


In [ ]:
# Define a multiplicative model
model_mult = LinearTrend(n_changepoints=25) ** (
    FourierSeasonality(period=365.25, series_order=10)
    + FourierSeasonality(period=7, series_order=3)
)

print(f"Model: {model_mult}")

In [ ]:
# Fit the multiplicative model
model_mult.fit(train)
print("Multiplicative model fitted!")

We plot the results to show how the multiplicative seasonality better captures the increase of variance with the trend.

In [ ]:
# Predict
future_mult = model_mult.predict(horizon=365, freq="D")

# Plot results
plt.figure(figsize=(14, 5))
plt.plot(train["ds"], train["y"], "b.", label="Training data", markersize=3)
plt.plot(test["ds"], test["y"], "g.", label="Test data", markersize=3)
plt.plot(
    future_mult["ds"], future_mult["yhat_0"], "r-", label="Prediction", linewidth=1
)
plt.title("Multiplicative Model: Air Passengers")
plt.xlabel("Date")
plt.ylabel("Number of Passengers")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
model_mult.plot(future_mult, y_true=test)
plt.tight_layout()
plt.show()

### Metrics comparison

We compare the standard metrics between the additive and the multiplicative models.

In [ ]:
metrics_additive = metrics(test, future_additive, "complete")
print("Additive Model Metrics:")
display(metrics_additive)

In [ ]:
metrics_mult = metrics(test, future_mult, "complete")
print("Multiplicative Model Metrics:")
display(metrics_mult)